In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sympy.physics.units as u
import sympy as sp
import numpy as np
import logging as log
import functools
import matplotlib.pyplot as plt
import seaborn
import pandas as pd
pd.set_option('display.max_columns', None)
%matplotlib inline
from IPython.display import display, HTML

import sys, os, importlib
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import miptlabs as ml
importlib.reload(ml)

importlib.reload(log)
log.basicConfig(level=log.INFO, format='LOG %(message)s', stream=sys.stdout)

sp.Basic.__str__=lambda expr, **settings:ml.PQStrPrinter(settings).doprint(expr)

sp.__version__

'1.1.2.dev'

In [3]:
data = pd.read_csv('10.4.csv')
display(data)

,#,num,type,I,B,Bfine,f
0,3.0,NaN,вода,1.79,234.911,226,10.0014
1,NaN,NaN,NaN,1.71,223.800,214,9.5286
2,NaN,NaN,NaN,1.85,239.090,230,10.1796
3,1.0,2794.0,резина,1.81,234.886,226,10.0000
4,NaN,NaN,NaN,1.87,240.890,230,10.2530
5,NaN,NaN,NaN,1.71,223.444,214,9.5132
6,2.0,2896.0,тефлон,1.95,235.067,239,10.0030


In [4]:

df1 = pd.DataFrame()

B = ml.get_column(data, 'B', lambda row: ml.PQ(row['B']*10**-3*u.tesla, sigma=0.1*10**-3*u.tesla))
Bfine = ml.get_column(data, 'Bfine', lambda row: ml.PQ(row['Bfine']*10**-3*u.tesla, sigma=1*10**-3*u.tesla))
f = ml.get_column(data, 'f', lambda row: ml.PQ(row['f']*10**6*u.hz, sigma=0.1*10**6*u.hz))

df1['#'] = data['#'].dropna().astype(int)
df1['type'] = data['type'].dropna()
df1['B'] = [ml.get_mean(B[0:3]), ml.get_mean(B[3:6]), B[6]]
df1['Bfine'] = [ml.get_mean(Bfine[0:3]), ml.get_mean(Bfine[3:6]), Bfine[6]]
df1['f'] = [ml.get_mean(f[0:3]), ml.get_mean(f[3:6]), f[6]]
display(df1)


,#,type,B,Bfine,f
0,3,вода,0.233±0.005 T (2.0%),0.223±0.005 T (2.2%),(99.00±2.20)*10^5 Hz (2.2%)
3,1,резина,0.233±0.005 T (2.2%),0.223±0.005 T (2.2%),(99.20±2.40)*10^5 Hz (2.4%)
6,2,тефлон,(2350.7±1.0)*10^-4 T (0.043%),0.2390±0.0010 T (0.42%),(100.00±1.00)*10^5 Hz (1.00%)


In [6]:
planck = 6.62607004*10**-34*u.m**2*u.kg/u.s
mu_N =  5.05078343*10**-27*u.joules/u.tesla
df1['g'] = ml.pqarray(
    ml.pqarray(
        [ml.get_mean((f/B)[0:3]),
         ml.get_mean((f/B)[3:6]),
         (f/B)[6]]) \
            * ((planck)/(mu_N))).repr_as(1)
S = 1/2
df1['mu'] = mu_N*df1['g']*S
df1['mu, mu_N'] = ml.pqarray(df1['g']*S).repr_as(1)
df1

,#,type,B,Bfine,f,g,mu,"mu, mu_N"
0,3,вода,0.233±0.005 T (2.0%),0.223±0.005 T (2.2%),(99.00±2.20)*10^5 Hz (2.2%),5.59±0.06 (1.0%),(141.10±1.40)*10^-28 J/T (1.0%),2.793±0.028 (1.0%)
3,1,резина,0.233±0.005 T (2.2%),0.223±0.005 T (2.2%),(99.20±2.40)*10^5 Hz (2.4%),5.58±0.06 (1.0%),(141.00±1.40)*10^-28 J/T (1.0%),2.792±0.028 (1.0%)
6,2,тефлон,(2350.7±1.0)*10^-4 T (0.043%),0.2390±0.0010 T (0.42%),(100.00±1.00)*10^5 Hz (1.00%),5.58±0.06 (1.0%),(141.00±1.40)*10^-28 J/T (1.0%),2.791±0.028 (1.0%)
